Please fill in your name and that of your teammate.

You:

Teammate:

# Introduction

Welcome to the twelfth lab. It's finally time to look closely at the foundations of Reinforcement Learning.

### How to pass the lab?

Below you find the exercise questions. Each question awarding points is numbered and states the number of points like this: **[0pt]**. To answer a question, fill the cell below with your answer (markdown for text, code for implementation). Incorrect or incomplete answers are in principle worth 0 points: to assign partial reward is only up to teacher discretion. Over-complete answers do not award extra points (though they are appreciated and will be kept under consideration). Save your work frequently! (`ctrl+s`)

**You need at least 14 points (out of 21 available) to pass** (66%).

# 1. Fundamentals

#### 1.1 **[1pt]** Explain the Reinforcement Learning paradigm in English. Use the words Environment, Agent, Action and Feedback.

#### 1.2 **[1pt]** Explain the equation for (pseudo-)Regret in English.

#### 1.3 **[1pt]** Explain in English the importance of _exploration vs. exploitation_ in MAB.

#### 1.4 **[1pt]** Explain what would happen if you had a discount factor $\gamma \gt 1$ in the Ice Maze example from the lecture.

#### 1.5 **[2pt]** The Bellman Equation is at the heart of the classical Reinforcement Learning framework. Write it below in Latex, then explain each of the terms.

# 2. Q-Learning

- The [OpenAI Gym](https://www.gymlibrary.dev/) maintains a broad set of Reinforcement Learning benchmarks. It is ready to import on Colab, or you can add to your local installation the environments we will use for this assignment using `pipenv install gym[toy_text,classic_control]`.
- You can find the Ice Maze (named Frozen Lake) [here](https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py).
- The Classic Control games are also relatively easy to solve, and much nicer to render. Feel free to give it a try. You can save a video with `env = gym.wrappers.Monitor(env, 'video', force = True)` right after the `gym.make()` call. More on this topic [here](https://hub.packtpub.com/openai-gym-environments-wrappers-and-monitors-tutorial/).
- To get you started, here is a working example of a random policy on the framework, which I took from their front page and customized.

#### 2.1 **[2pt]** Write a Python function `choose_action` that takes as arguments `Q`, `state`, `epsilon`, `actions` and returns an action chosen according to Q-values using an epsilon-greedy policy.

- `Q` is a dictionary that has states as keys. Each value is a numpy array with the Q-values corresponding to each action from the state used as key.
- You cannot initialize `Q` with all possible states, because they can be infinite or unreachable. Using `Q[state]` with missing key `state` will throw an error; use instead `Q.get(state)` then check if its return `is None`.
- `state` can be simply a number.
- `actions` is a list of possible actions between which to choose.

HINT: here is some code I used for testing `choose_action()` and expected outputs:

In [ ]:
Q = {1:[0,0,1], 2:[0,1,0]}
for _ in range(50):
    print(choose_action(Q, 1, 0.0, []), end='') #=> 2
print()
for _ in range(50):
    print(choose_action(Q, 2, 0.0, []), end='') #=> 1
print()
for _ in range(50):
    print(choose_action(Q, 2, 1.0, range(3)), end='') #=> random action

#### 2.2 **[2pt]** Write a Python function `update_Q` that takes as arguments `Q`, `state`, `action`, `next_state`, `reward` and `num_actions`, and updates the `Q` dictionary according to Q-Learning. 

- Here you will need to initialize `Q[state]` if previously unexplored.
- The future expected reward is the highest Q value from the next state -- or zero if unexplored. That's what the last argument is for.
- I'll leave the testing to you here. You may feel like just running the next question for testing (called _integration testing_ ), but testing it in isolation ( _unit testing_ ) is a much more controlled verification which should not be skipped (i.e. _easier_ ).

#### 2.3 **[1pt]** Run the code below to successfully use Q-Learning to solve the OpenAI Gym `FrozenLake` environment.

- The code below is already correct, it just needs the two functions from above.
- If you got both previous answers right, yes this is a free extra point for you :) just run it to produce the outputs.
- If this does not run correctly, something is likely wrong with your implementation above. Go back and test a bit more.
- The parameters should work as they are, but feel free to play with them and make sure by the last epochs the environment is solved more often than not.

In [ ]:
# Initialization and settings
env = gym.make("FrozenLake-v1", is_slippery=False)
num_episodes = 1000
max_nsteps = 30
gamma = 0.9
alpha = 0.6
epsilon = 0.3

num_actions = env.action_space.n
Q = {} # hashing states to per-action reward arrays

# Loop for each episode
for ith_episode in range(num_episodes):

    # Reset the environment (and obtain first observation)
    state, _ = env.reset()
    total_reward = 0

    # For each timestep
    for t in range(max_nsteps):
        
        # Choose action according to Q-values using epsilon-greedy policy
        # THIS SHOULD USE YOUR IMPLEMENTATION
        action = choose_action(Q, state, epsilon, range(num_actions))
        
        # Execute action: get reward, move env to next state
        next_state, reward, done, truncated, info = env.step(action)
        # Add negative reward for falling in hole
        if done and reward == 0: reward = -1

        # Some useful printing to verify progression - (W)in or (l)ose
        if reward == -1: print('l', end='', flush=True)
        if reward == 1: print('W', end='', flush=True)

        # Accumulate reward. Note this environment only rewards on termination though.
        total_reward += reward
        
        # Update Q function for current state and action
        # THIS SHOULD USE YOUR IMPLEMENTATION
        update_Q(Q, state, action, next_state, reward, num_actions, alpha, gamma)
        
        # Update internal state
        state = next_state
        
        # Terminate if episode ended
        if done: break

#### 2.4 **[1pt]** Write a Python code snippet that satisfies the following requirements: (i) runs a fully-greedy policy, (ii) using the Q-values learned in question 2.3, (iii) renders the environment in each step, and (iv) write less than 10 lines of code.

- The point of this question is for you to go back to 2.3, study it and understand it. If you get how it works, this snippet will take but a second.
- For the subpoint (iii) you should look at the method `render()` of the OpenAI environment.

#### 2.5 **[1pt]** Modify the code from question 2.3 to record the cumulative reward and number of time-steps for each episode. Then plot them with two line plots using Seaborn.

- Think carefully: you need to augment the code above, by first initializing a proper data structure, then recording the statistics at the end of each episode. Finally plot them below.
- If the plot looks uniform while you expect improvement, try re-running it a few times. Remember that the exploration capabilities of this methods are pretty limited, so short runs (such as this) depend significantly on the initialization conditions (i.e. luck).

# 3. DQN

- DQN basically implements Q-Learning but uses a (deep) neural network to learn the rewards (with a few tricks). The main advantage versus the dictionary-based approach above is *generalization*: the network can output an expected reward for **all actions**, even those that were not yet explored.
- This code takes a while to execute. You may want to use [TPUs with Colab](https://www.tensorflow.org/guide/tpu)
  - Though if the `env.render()` does not work in that case, just comment it out
  - You can use the `render` option to save a video instead, find it in your Google Drive
- This code was originally taken from [this DQN tutorial](https://towardsdatascience.com/reinforcement-learning-w-keras-openai-dqns-1eed3a5338c) (with few edits). Study it to answer the following questions.
- Learn more about the CartPole [here](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py).

In [ ]:
## IMPLEMENTATION ##

import gym
import numpy as np
import random

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

# Do you know what a deque is? https://en.wikipedia.org/wiki/Deque
# Though here it's used just to cap its number of elements
from collections import deque

class DQN:
    def __init__(self, env):
        self.env     = env
        self.memory  = deque(maxlen=2000) # after 2000 will delete oldest record(s)

        self.gamma = 0.85
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.01
        self.tau = .125
        self.batch_size = 64 # try different batch sizes
        
        self.model        = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()
        state_shape  = self.env.observation_space.shape
        model.add(Dense(24, activation="relu"))
        model.add(Dense(48, activation="relu"))
        model.add(Dense(24, activation="relu"))
        model.add(Dense(self.env.action_space.n))
        model.compile(loss="mean_squared_error",
            optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() > self.epsilon:
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state).flatten())

    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        # This trick records past experience and re-plays for accelerate the training
        if len(self.memory) < self.batch_size: return
        samples = random.sample(self.memory, self.batch_size)
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state).flatten())
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state, target, epochs=1, verbose=0)

    # This part can be confusing: can you explain what is happening line by line?
    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)        

In [ ]:
## MAIN ##

# Your Python kernel can crash if you redefine your `env` variable without closing
# the gym environment properly. But also crashes if you call a method on `env` if
# it has not been defined yet. The `try except` construct in Python allows you to 
# literally try to execute some code while catching errors and exceptions if
# something goes wrong. Search online for what a NameError is!
try: env.close()
except NameError: pass # `pass` literally does nothing

env     = gym.make("CartPole-v1")
gamma   = 0.9
# Sometimes you will find code that uses (1-epsilon) for epsilon
# Other times you will find a _decaying_ epsilon, lowering over time
epsilon = .5 # it was .9 originally, play with it and understand its role

ntrials   = 5 # I can see the beginning of learning with 20, feel free to raise this
max_nstep = 200 # This environment is capped at 200 anyway, but you can try shorter

dqn_agent = DQN(env=env)
for trial in range(ntrials):
    print(f"Trial {trial+1}:")
    cur_state, _ = env.reset() # never forget to reset the env
    cur_state = cur_state.reshape(1, -1)
    tot_reward = 0
    for step in range(max_nstep):
        # print('.', end='', flush=True)
        action = dqn_agent.act(cur_state)
        print(action, end='', flush=True)
        
        new_state, reward, done, _, _ = env.step(action)
        if done: reward = -100 # It's much easier to learn if you punish failing
        tot_reward += reward

        new_state = new_state.reshape(1,-1)
        dqn_agent.remember(cur_state, action, reward, new_state, done)

        dqn_agent.replay()       # Internally iterates default (prediction) model
        dqn_agent.target_train() # What does this do?
        
        if done: break
    print(f" Reward: {tot_reward}")

env.close()

# Here is how to save trained Keras models
dqn_agent.model.save("trained.model")

#### 3.1 **[3pt]** There are three bugs in the implementation. Fix them until it runs successfully.

- One is in `## IMPLEMENTATION ##`, easy to find because it stops it from running.
- One is just harder, messes with the policy. It's also in `## IMPLEMENTATION ##` but there's a hint in `## MAIN ##`.
- One is a missing line in `## MAIN ##`: something there makes no sense.
- Of course you can just look at the original implementation. But you will learn less and will not be able do it at the exam, so give it a fair try first, because it is a fair exercise.

#### 3.2 **[1pt]** Compare the shape of the model used with that of an autoencoder. Give one reason as to why they do or do not look alike.

#### 3.3 **[1pt]** How many inputs and outputs neurons does the model have? Write the code that answers the question. 

#### 3.4 **[3pt]** Write Python code to (i) create a new DQN agent, (ii) load a trained model into the agent's `model`, (iii) change the agent's epsilon-related parameters to enforce a greedy policy, and (iv) runs the greedy policy rendering each frame.

- Remember to instantiate the environment first, and to close it at the end, or you may have problems.
- Loading the model is easy -- if you saved it to a file in the `main` cell.
- To enforce the greedy policy you need to set three agent variables, to sensible values.
- You saw a similar evaluation loop when implementing Q-Learning. Just double check the differences with the DQN implementation to make sure it runs. You need to use `env.render()` here (if you're local; if you work on Colab just write a comment about it or save a video).
- You can use `time.sleep()` to slow down the loop if the rendering is too fast. Remember also that the env's rendering window will close when you close the environment, so a pause there can also help.

# At the end of the exercise

Bonus question with no points! Answering this will have no influence on your scoring, not at the assignment and not towards the exam score -- really feel free to ignore it with no consequence. But solving it will reward you with skills that will make the next lectures easier, give you real applications, and will be good practice towards the exam.

The solution for this questions will not be included in the regular lab solutions pdf, but you are welcome to open a discussion on the Moodle: we will support your addressing it, and you may meet other students that choose to solve this, and find a teammate for the next assignment that is willing to do things for fun and not only for score :)

#### BONUS **[ZERO pt]** Solve another of the [OpenAI Gym environments](https://gym.openai.com/envs/#classic_control) by copying and modifying the above DQN implementation. This is very useful to learn to use different environments, and to see first-hand the limits of DRL. Can you imagine why the Mountain Car is hard?

#### BONUS **[ZERO pt]** Augment the DQN implementation to track episode size and cumulative reward and plot them with lineplots.

### Final considerations

- Reinforcement Learning is both the name of the learning paradigm and of the framework classically used to address such problems.
- The amount of research currently dedicated to solve RL problems by improving both DL and the RL framework is staggering. Results still keep coming, but at a much slower pace than in purer SL applications (in front of orders of magnitude more investment).
- Next week we will explore the limitations of the RL framework, and beyond.